<a href="https://colab.research.google.com/github/qwiksilva/cs224w-github-rec/blob/master/srw_feateng_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!pip install chars2vec

     |████████████████████████████████| 8.1MB 2.8MB/s 
  Created wheel for chars2vec: filename=chars2vec-0.1.7-cp36-none-any.whl size=8111095 sha256=a21de6ee9ef962a22f67788ed8427825b5ba6f3b3bf8283c6fa92cad7f1eebb5
  Stored in directory: /root/.cache/pip/wheels/97/b6/65/d7e778ef1213ec77d315aea0f536068b96e36cc94c02abbfde
Successfully built chars2vec


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import chars2vec
c2v = chars2vec.load_model('eng_100')

In [0]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import joblib

from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
from absl import logging
# Reduce logging output.
logging.set_verbosity(logging.ERROR)

%matplotlib inline
sns.set(color_codes=True)

home = "/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019"

In [0]:
filepath_df = pd.read_csv(os.path.join(home, 'final_filepath_df_12062019.csv'), index_col=0)
filepath_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149858 entries, 13986 to 38381
Data columns (total 4 columns):
additions    149858 non-null int64
changes      149858 non-null int64
deletions    149858 non-null int64
filename     149858 non-null object
dtypes: int64(3), object(1)
memory usage: 5.7+ MB


In [0]:
comments_df = pd.read_csv(os.path.join(home, 'final_comments_df_12062019.csv'), index_col=0, parse_dates=['pr_created_at', 'comment_created_at'])
print(comments_df.pr_id.nunique())

# Drop extra bots
extra_bot_id = []
for _, (id, name) in comments_df.loc[:, ['commenter_id', 'commenter_login']].drop_duplicates().iterrows():
    if 'k8s-' in name or 'bot' in name:
        print(name, id)
        extra_bot_id.append(id)
comments_df.drop(comments_df[comments_df.commenter_id.isin(extra_bot_id)].index, inplace=True)
print(comments_df.pr_id.nunique())

# Only consider PR in filepath_df
comments_df = comments_df.loc[comments_df.pr_id.isin(filepath_df.index.values)].copy()
print(comments_df.shape)

27750
k8s-oncall 13734568
k8s-reviewable 24305385
k8s-slack 14020429
27750
(384665, 10)


In [0]:
# Filter train/test

start_train = '2018-01-01'
start_test = '2018-06-01'
end_test = '2019-01-01'

train_filter = (comments_df.comment_created_at >= start_train) & (comments_df.comment_created_at < start_test) & (comments_df.pr_created_at >= start_train) & (comments_df.pr_created_at < start_test)
test_filter = (comments_df.comment_created_at >= start_test)  & (comments_df.comment_created_at < end_test) & (comments_df.pr_created_at >= start_test) & (comments_df.pr_created_at < end_test)

train_comments_df = comments_df.loc[train_filter].copy()
test_comments_df = comments_df.loc[test_filter].copy()
print(train_comments_df.shape, test_comments_df.shape)

# More clean
bad_train_pr_list_in_2018 = joblib.load(os.path.join(home, 'bad_train_pr_list_in_2018.joblib'))
bad_test_pr_list_in_2018 = joblib.load(os.path.join(home, 'bad_test_pr_list_in_2018.joblib'))  
train_comments_df.drop(train_comments_df[train_comments_df.pr_id.isin(bad_train_pr_list_in_2018)].index, inplace=True) # Bad no non-owner commenters
test_comments_df.drop(test_comments_df[test_comments_df.pr_id.isin(bad_test_pr_list_in_2018)].index, inplace=True)
test_comments_df = test_comments_df.loc[test_comments_df.commenter_id.isin(train_comments_df.commenter_id.unique())].copy() # No new users entering the graph
print(train_comments_df.shape, test_comments_df.shape)

(31153, 10) (44001, 10)
(31064, 10) (37927, 10)


In [0]:
train_user_list = set(train_comments_df.commenter_id.unique().tolist())
test_user_list = set(test_comments_df.commenter_id.unique().tolist())
train_pr_list = set(train_comments_df.pr_id.unique().tolist())
test_pr_list = set(test_comments_df.pr_id.unique().tolist())
print(len(train_user_list), len(test_user_list), len(train_pr_list), len(test_pr_list))

690 366 2719 3837


In [0]:
def generate_pr_user_feat_df(core_df):
    combine = [core_df.groupby(['commenter_id', 'pr_id'])['comment_created_at'].min(),
               core_df.groupby(['commenter_id', 'pr_id'])['comment_created_at'].max(),
               core_df.groupby(['commenter_id', 'pr_id'])['comment'].count()]
    colnames = ['first_comment', 'last_comment', 'num_comment']
    pr_user_feat_df = pd.concat(combine, axis=1)
    pr_user_feat_df.columns = colnames
    user_gb_list = [core_df.groupby(['user_id', 'pr_id'])['username'].nunique().groupby('user_id').count(),
                    core_df.groupby(['commenter_id', 'pr_id'])['commenter_login'].nunique().groupby('commenter_id').count()]
    user_df = pd.concat(user_gb_list, axis=1)
    user_df.columns = ['num_pr_submitted', 'num_pr_commented']
    user_df.rename_axis('commenter_id')
    pr_user_feat_df = pr_user_feat_df.join(user_df, on='commenter_id').fillna(value=0)
    pr_user_feat_df = pr_user_feat_df.join(core_df.groupby('pr_id')['pr_created_at'].max(), on='pr_id')
    pr_user_feat_df['first_comment_age'] = (pr_user_feat_df['first_comment'] - pr_user_feat_df['pr_created_at']).apply(lambda x: x.total_seconds()/86400)
    pr_user_feat_df['last_comment_age'] = (pr_user_feat_df['last_comment'] - pr_user_feat_df['pr_created_at']).apply(lambda x: x.total_seconds()/86400)
    return pr_user_feat_df

train_pr_user_df = generate_pr_user_feat_df(train_comments_df)
test_pr_user_df = generate_pr_user_feat_df(test_comments_df)
train_pr_owner_dict = train_comments_df.groupby('pr_id')['user_id'].min()
test_pr_owner_dict = test_comments_df.groupby('pr_id')['user_id'].min()

In [0]:
def generate_evaluation_dict(pr_user_df, owner_lookup):
    out = defaultdict(list)
    for (commenter_id, pr_id), last_comment_ts in pr_user_df['last_comment'].iteritems():
        if commenter_id != owner_lookup[pr_id]:
            out[pr_id].append((commenter_id, last_comment_ts))
    for pr_id, pr_value in out.items():
        out[pr_id] = [user_id for user_id, ts in sorted(pr_value, key=lambda x: x[1])]
    return out

train_dict = generate_evaluation_dict(train_pr_user_df, train_pr_owner_dict)
test_dict = generate_evaluation_dict(test_pr_user_df, test_pr_owner_dict)

# bad_train_pr_list_in_2018 = train_pr_list - train_dict.keys() # No non-owner commenters
# print(len(bad_train_pr_list_in_2018))
# joblib.dump(bad_train_pr_list_in_2018, os.path.join(home, 'bad_train_pr_list_in_2018.joblib'))
# bad_test_pr_list_in_2018 = test_pr_list - test_dict.keys()
# print(len(bad_test_pr_list_in_2018))
# joblib.dump(bad_test_pr_list_in_2018, os.path.join(home, 'bad_test_pr_list_in_2018.joblib'))

In [0]:
filepath_df = filepath_df.loc[train_pr_list | test_pr_list, :].copy()

In [0]:
# Compile description
lookup = train_comments_df.groupby('pr_id')['pr_created_at'].max()
title_list = []
for pr_id, ts in tqdm(lookup.iteritems()):
    title = train_comments_df.loc[(train_comments_df.pr_id==pr_id) & (train_comments_df.pr_created_at==ts), 'title'].values[0]
    title_list.append(title)
train_title_df = lookup.reset_index()
train_title_df['title'] = title_list

lookup = test_comments_df.groupby('pr_id')['pr_created_at'].max()
title_list = []
for pr_id, ts in tqdm(lookup.iteritems()):
    title = test_comments_df.loc[(test_comments_df.pr_id==pr_id) & (test_comments_df.pr_created_at==ts), 'title'].values[0]
    title_list.append(title)
test_title_df = lookup.reset_index()
test_title_df['title'] = title_list

print(train_title_df.shape, test_title_df.shape)
title_df = pd.concat([train_title_df, test_title_df]).set_index('pr_id').drop_duplicates()
print(title_df.shape)

process = title_df.groupby('pr_id')['title'].unique()
def combine(title_list):
    out = []
    for title in title_list:
        words = title.lower().split()
        out.extend(words)
    return ' '.join(out)
final_title_compilation = process.apply(lambda x: combine(x))

2719it [00:04, 620.69it/s]
3837it [00:06, 619.57it/s]


(2719, 3) (3837, 3)
(6556, 2)


In [0]:
# Run char2vec on filepath
pr_filepath_list = filepath_df.loc[:, 'filename'].values
split_out = []
for filepath in tqdm(pr_filepath_list):
    components = filepath.split('/')
    vec = c2v.vectorize_words(components)
    split_out.append(np.mean(vec, axis=0))
filepath_df['c2v_filepath'] = split_out

# Compile PR features for cosine similarity
filepath_summary_df = pd.concat([filepath_df.groupby('pr_id')['additions', 'changes', 'deletions'].mean(), 
                                 filepath_df.groupby('pr_id')['c2v_filepath'].apply(lambda x: np.mean(x, axis=0)),
                                 filepath_df.groupby('pr_id')['filename'].count(),
                                 final_title_compilation], axis=1)
filepath_summary_df.columns = ['average_additions', 'average_changes', 'average_deletions', 'average_c2v_filepath', 'filepath_count', 'title']

# Run char2vec on title
title_list = filepath_summary_df.loc[:, 'title'].values
split_out = []
for filepath in tqdm(title_list):
    components = filepath.split()
    vec = c2v.vectorize_words(components)
    split_out.append(np.mean(vec, axis=0))
filepath_summary_df['c2v_title'] = split_out

In [0]:
del c2v

In [0]:
from sklearn.preprocessing import StandardScaler

def generate_pr_feat_matrix(pr_id_list, filepath_summary_df, train=True, scaler_dict={}):
    col_list = ['average_additions', 'average_changes', 'average_deletions', 'average_c2v_filepath', 'filepath_count', 'c2v_title']
    extract = filepath_summary_df.loc[pr_id_list, col_list]
    print(extract.shape)
    collection = []
    for feat_name in extract.columns:
        column = extract.loc[:, feat_name].values
        if 'c2v' in feat_name:
            column = np.vstack(column.tolist())
        else:
            if train:
                scaler = StandardScaler()
                column = scaler.fit_transform(column.reshape(-1, 1)) # Potential leakage
                scaler_dict[feat_name] = scaler
            else:
                column = scaler_dict[feat_name].transform(column.reshape(-1, 1))
        collection.append(column)
    pr_feat_matrix = np.hstack(collection)
    pr_list = extract.index.values.tolist() # Match up with cosine
    pr_cosine_index_map = dict(zip(pr_list, range(len(pr_list))))
    return pr_feat_matrix, pr_cosine_index_map, scaler_dict

In [0]:
%%time
pr_list = set(train_pr_list) | set(test_pr_list)
pr_feat_matrix, pr_cosine_index_map, scaler_dict = generate_pr_feat_matrix(pr_list, filepath_summary_df)
print(pr_feat_matrix.shape)

(6556, 6)
(6556, 204)
CPU times: user 38.3 ms, sys: 1.98 ms, total: 40.3 ms
Wall time: 40.3 ms


In [0]:
# filepath_summary_df = pd.read_csv(os.path.join('/gdrive/My Drive/Colab Notebooks/cs224w-data/', 'fp_summary_1209.csv'), index_col=0)
# pr_feat_matrix = joblib.load(os.path.join('/gdrive/My Drive/Colab Notebooks/cs224w-data/', 'pr_feat_matrix_12092019.joblib'))
# pr_cosine_index_map = joblib.load(os.path.join('/gdrive/My Drive/Colab Notebooks/cs224w-data/', 'pr_cosine_index_map_12092019.joblib'))
# joblib.dump(pr_feat_matrix, 'pr_feat_matrix_12092019.joblib')
# joblib.dump(pr_cosine_index_map, 'pr_cosine_index_map_12092019.joblib')

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_matrix = cosine_similarity(pr_feat_matrix, pr_feat_matrix) # Take a lot of RAM here, need to crash to ask Colab for 25GB

# Split half if crash
# half = range(pr_feat_matrix.shape[0]//2)
# other = range(pr_feat_matrix.shape[0]//2, pr_feat_matrix.shape[0])
# print(half, other)
# half_cosine_sim_matrix = cosine_similarity(pr_feat_matrix[half, ], pr_feat_matrix)
# other_cosine_sim_matrix = cosine_similarity(pr_feat_matrix[other, ], pr_feat_matrix)

In [0]:
from collections import defaultdict
from scipy import sparse
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

def generate_cosine_sim_feat(similarity_matrix, pr_index_map, pr_user_feat_df, feat_name):
    users_to_pr = defaultdict(list)
    pr_to_users = defaultdict(list)
    for commenter_id, pr_id in pr_user_feat_df.index:
        users_to_pr[commenter_id].append(pr_id)
        pr_to_users[pr_id].append(commenter_id)

    out = []
    for user_id, source_pr_id in tqdm(pr_user_feat_df.index):
        interacted_pr_list = users_to_pr[user_id]
        if len(interacted_pr_list) == 0: 
            print(user_id)
        elif len(interacted_pr_list) == 1: 
            out.append(0.0)
        else:
            cumm = []
            for target_pr_id in interacted_pr_list:
                if source_pr_id != target_pr_id:
                    sim = similarity_matrix[pr_index_map[source_pr_id], pr_index_map[target_pr_id]]
                    cumm.append(sim)
            # if len(cumm) == 0:
            #     print(user_id, source_pr_id, target_pr_id, interacted_pr_list)
            out.append(sum(cumm)/len(cumm))
    pr_user_feat_df[feat_name] = out

    return pr_user_feat_df
    

def convert_adj_matrix(mat, row_map, column_map, row_name, col_name):
    m, n = mat.shape
    print("Number of %s = %d" %(row_name, m))
    print("Number of %s = %d" %(col_name, n))
    out = sparse.lil_matrix((m+n, m+n), dtype=type(mat[0, 0]))
    # out = np.zeros((m+n, m+n), dtype=type(mat[0, 0])) # Sparse matrix, lower dtype

    out[:m, m:] = mat
    out[m:, :m] = mat.T
    map_values = range(m + n)
    map_keys = list(zip(list(row_map.keys()), [row_name]*len(row_map))) + list(zip(list(column_map.keys()), [col_name]*len(column_map)))
    total_map = dict(zip(map_keys, map_values))
    return out, total_map


def process_sparse_matrix(feat, pr_user_feat_df):
    series = pr_user_feat_df.loc[:, feat]

    user_list = series.reset_index().commenter_id.unique().tolist()
    pr_list = series.reset_index().pr_id.unique().tolist()
    user_index_map = dict(zip(user_list, range(len(user_list))))
    pr_index_map = dict(zip(pr_list, range(len(pr_list))))

    # out = np.zeros((len(pr_index_map), len(user_index_map)), dtype=)
    out = sparse.lil_matrix((len(pr_index_map), len(user_index_map)), dtype=type(series[0]))
    for (user_id, pr_id), feat_value in series.iteritems():
        out[pr_index_map[pr_id], user_index_map[user_id]] = feat_value

    return convert_adj_matrix(out, pr_index_map, user_index_map, "PR", "User")


def main(pr_user_feat_df, train=True, scaler_dict=None):
    global cosine_sim_matrix, pr_cosine_index_map
    cosine_feat_name = 'neighbor_PR_average_cosine_sim'
    pr_user_feat_df = generate_cosine_sim_feat(cosine_sim_matrix, pr_cosine_index_map, pr_user_feat_df, cosine_feat_name)

    selected = {
        'num_comment': np.int16, 
        'num_pr_submitted': np.int16,
        'num_pr_commented': np.int16,
        'first_comment_age': np.float32,
        'last_comment_age': np.float32,
        cosine_feat_name: np.float32,
    }
    pr_user_feat_df = pr_user_feat_df.loc[:, selected.keys()].astype(selected).copy()

    def rescale(colname, df, scaler=None):
        if not scaler:
            scaler = StandardScaler()
            df.loc[:, colname] = scaler.fit_transform(df.loc[:, colname].values.reshape(-1, 1)).reshape(-1)
        else:
            df.loc[:, colname] = scaler.transform(df.loc[:, colname].values.reshape(-1, 1)).reshape(-1)
        return scaler

    srw_output = {}
    node_mapping = None
    if train:
        train_scalers = {} 
    for feat in selected.keys():
        print(feat, pr_user_feat_df.loc[:, feat].mean(), pr_user_feat_df.loc[:, feat].std())
        if feat != cosine_feat_name:
            if train:
                train_scalers[feat] = rescale(feat, pr_user_feat_df)
            else:
                _ = rescale(feat, pr_user_feat_df, scaler_dict[feat])
        print(feat, pr_user_feat_df.loc[:, feat].mean(), pr_user_feat_df.loc[:, feat].std())
        output, node_mapping = process_sparse_matrix(feat, pr_user_feat_df)
        srw_output[feat] = output
        # joblib.dump(output, '%s_sparse_lil.joblib' %feat)

    if train:
        return srw_output, node_mapping, train_scalers
    else:
        return srw_output, node_mapping

In [0]:
train_srw_output, train_node_mapping, train_scalers = main(train_pr_user_df, train=True)
for feat_name, sparse_mat in train_srw_output.items():
    print(feat_name, sparse_mat.shape)

100%|██████████| 9153/9153 [00:00<00:00, 17239.40it/s]


num_comment 3.393859936632798 4.409555527810614
num_comment 4.1898092119088995e-16 1.000054631374828
Number of PR = 2719
Number of User = 690
num_pr_submitted 21.608106631705454 23.324147872122925
num_pr_submitted -1.4220145994966702e-15 1.0000546313748258
Number of PR = 2719
Number of User = 690
num_pr_commented 112.15415710695947 126.1668109762065
num_pr_commented -1.1223647622811139e-15 1.0000546313747958
Number of PR = 2719
Number of User = 690
first_comment_age 4.455038070678711 10.988587379455566
first_comment_age 5.446339557124702e-08 1.0000513792037964
Number of PR = 2719
Number of User = 690
last_comment_age 8.868515014648438 15.646407127380371
last_comment_age -3.46765780534497e-08 1.0000548362731934
Number of PR = 2719
Number of User = 690
neighbor_PR_average_cosine_sim 0.5762746334075928 0.14846810698509216
neighbor_PR_average_cosine_sim 0.5762746334075928 0.14846810698509216
Number of PR = 2719
Number of User = 690
num_comment (3409, 3409)
num_pr_submitted (3409, 3409)
num

In [0]:
test_srw_output, test_node_mapping = main(test_pr_user_df, train=False, scaler_dict=train_scalers)
for feat_name, sparse_mat in test_srw_output.items():
    print(feat_name, sparse_mat.shape)

100%|██████████| 12313/12313 [00:01<00:00, 10703.96it/s]


num_comment 3.080240396329083 4.217802419761555
num_comment -0.07112659581952308 0.9565664424688908
Number of PR = 3837
Number of User = 366
num_pr_submitted 28.631527653699344 30.601134587218887
num_pr_submitted 0.3011387494046489 1.3120653554872068
Number of PR = 3837
Number of User = 366
num_pr_commented 183.6388370015431 199.45433255639102
num_pr_commented 0.5666195780632681 1.5809643398089193
Number of PR = 3837
Number of User = 366
first_comment_age 5.807437419891357 14.945874214172363
first_comment_age 0.12308144569396973 1.36019766330719
Number of PR = 3837
Number of User = 366
last_comment_age 10.727508544921875 21.2080078125
last_comment_age 0.11882064491510391 1.3555282354354858
Number of PR = 3837
Number of User = 366
neighbor_PR_average_cosine_sim 0.5818747878074646 0.10965071618556976
neighbor_PR_average_cosine_sim 0.5818747878074646 0.10965071618556976
Number of PR = 3837
Number of User = 366
num_comment (4203, 4203)
num_pr_submitted (4203, 4203)
num_pr_commented (4203, 

In [0]:
joblib.dump(train_srw_output, os.path.join(home, 'train_srw_output_v4_2018_only.joblib'))
joblib.dump(train_node_mapping, os.path.join(home, 'train_node_mapping_v4_2018_only.joblib'))
joblib.dump(test_srw_output, os.path.join(home, 'test_srw_output_v4_2018_only.joblib'))
joblib.dump(test_node_mapping, os.path.join(home, 'test_node_mapping_v4_2018_only.joblib'))

['/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019/test_node_mapping_v4_2018_only.joblib']

In [0]:
joblib.dump(train_comments_df, os.path.join(home, 'train_comments_df_v4_2018_only.joblib'))
joblib.dump(test_comments_df, os.path.join(home, 'test_comments_df_v4_2018_only.joblib'))

['/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019/test_comments_df_v4_2018_only.joblib']

In [0]:
joblib.dump(filepath_summary_df, os.path.join(home, 'filepath_feat_df_v4_2018_only.joblib'))

['/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019/filepath_feat_df_v4_2018_only.joblib']

In [0]:
def dump_stuff(obj, fname):
    return joblib.dump(obj, os.path.join(home, '%s_v4_2018_only.joblib'%fname))

In [0]:
dump_stuff(train_dict, 'train_dict')

['/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019/train_dict_v4_2018_only.joblib']

In [0]:
dump_stuff(test_dict, 'test_dict')

['/gdrive/My Drive/Colab Notebooks/cs224w-data/final-data-12062019/test_dict_v4_2018_only.joblib']